 # Relation between Stock Revenue and share price

We will use WebScraping Plus Yfinance to Analyze the relation between Stock Revenue and share price for two stocks, GameStop and Apple.

In [27]:
!pip install yfinance==0.1.67
#!pip install pandas==1.3.3
!pip install requests==2.26.0
!pip install beautifulsoup4==4.10.0 
!pip install plotly==5.3.1

In [29]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function


We define the function `make_graph`. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.


In [30]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

 ##  yfinance to Extract  APPLE Stock Data



The stock is Tesla and its ticker symbol is TSLA.


In [49]:
APPLE = yf.Ticker('AAPL')


In [50]:

APPLE_data = APPLE.history( period='max')

In [51]:
APPLE_data.reset_index(inplace=True)
APPLE_data =APPLE_data[['Date','Close']] ##we just want these two columns
APPLE_data.head()

,Date,Close
0,1980-12-12,0.100039
1,1980-12-15,0.094820
2,1980-12-16,0.087861
3,1980-12-17,0.090035
4,1980-12-18,0.092646


##  yfinance to Extract  GameStop Stock Data


now we do the same thing for GameStop. The stock is GameStop and its ticker symbol is GME.

In [45]:
GameStop = yf.Ticker('GME')
gme_data= GameStop.history( period='max')
gme_data.reset_index(inplace=True)
gme_data = gme_data[['Date','Close']]##we just want these two columns

##  Webscraping to Extract APPLE Revenue Data


 webpage https://www.macrotrends.net/stocks/charts/AAPL/apple/revenue.

In [52]:
url = "https://www.macrotrends.net/stocks/charts/AAPL/apple/revenue"


In [55]:
read_html_pandas_data = pd.read_html(url)
apple_revenue = read_html_pandas_data[1]
new_column = ['Date','Revenue']
apple_revenue.columns = new_column
apple_revenue.head()

,Date,Revenue
0,2022-06-30,"$82,959"
1,2022-03-31,"$97,278"
2,2021-12-31,"$123,945"
3,2021-09-30,"$83,360"
4,2021-06-30,"$81,434"


In [56]:
#removing the comma and dollar sign from the `Revenue` column
apple_revenue["Revenue"] = apple_revenue['Revenue'].str.replace(',|\$',"")

/var/folders/bg/nwrfqr3x53x6dwwjtgkf3bl80000gn/T/ipykernel_4896/933762540.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.



In [57]:
#removing an null or empty strings in the Revenue column
apple_revenue.dropna(inplace=True)

apple_revenue = apple_revenue[apple_revenue['Revenue'] != ""]

In [58]:
apple_revenue.tail()

,Date,Revenue
49,2010-03-31,13499
50,2009-12-31,15683
51,2009-09-30,12207
52,2009-06-30,9734
53,2009-03-31,9084


##  Webscraping to Extract GME Revenue Data


 webpage <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html>


In [17]:
url2="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"


In [18]:
read_html_pandas_data = pd.read_html(url2)
gme_revenue = read_html_pandas_data[1]
new_column = ['Date','Revenue']
gme_revenue.columns = new_column
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")

/var/folders/bg/nwrfqr3x53x6dwwjtgkf3bl80000gn/T/ipykernel_4896/2655292544.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")


In [19]:
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


##  Plot Apple Stock Graph (share price, Revenue)


In [60]:
make_graph(APPLE_data, apple_revenue, 'Apple')

##   GameStop Stock Graph (share price, Revenue)


In [42]:
make_graph(gme_data, gme_revenue, 'GameStop')